IMPORT OF USED LIBRARIES

In [10]:
import pandas as pd
from datetime import datetime, timedelta

LOADED OUTPUT DATASET

In [51]:
path_to_raw_output_data = "data/RozsAgloSP4-2018-vystup.xlsm"

data_output = pd.read_excel(path_to_raw_output_data)

data_output

,Dátum a čas,Zmena,8hod,12hod,SP,Vzorka,Zhoda,0-5,2024-10-05 00:00:00,10-25,nad 25,nad 10,ISO Zhoda,ISO 0-5,"ISO > 6,3"
0,2018-12-31 12:08:18,1,1D,R7,4,41679,A,10.71,21.43,42.50,25.36,67.86,NaN,NaN,NaN
1,2018-12-30 23:56:11,3,3E,N9,4,41676,A,6.90,20.69,37.93,34.48,72.41,NaN,NaN,NaN
2,2018-12-30 12:43:11,1,1D,R8,4,41673,A,8.05,22.88,38.98,30.08,69.06,NaN,NaN,NaN
3,2018-12-30 00:15:03,3,3B,N5,4,41670,A,6.21,18.63,46.58,28.57,75.15,NaN,NaN,NaN
4,2018-12-29 12:23:16,1,1A,R9,4,41667,A,4.21,18.95,42.11,34.74,76.85,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,2018-01-02 11:30:07,1,1B,R5,4,40004,N,36.17,21.28,26.60,15.96,42.56,NaN,NaN,NaN
656,2018-01-02 02:02:57,3,3D,N7,4,40003,A,11.62,30.71,29.05,28.63,57.68,NaN,NaN,NaN
657,2018-01-01 16:49:22,2,2C,R6,4,40002,A,9.46,16.55,40.19,33.80,73.99,NaN,NaN,NaN
658,2018-01-01 10:20:53,1,1B,R6,4,40001,A,9.19,21.78,36.75,32.28,69.03,NaN,NaN,NaN


In [52]:
adjusted_times = []

# Loop through each time stamp in the 'Dátum a čas' column of data_output
for time_stamp in data_output['Dátum a čas']:

    # Convert time stamp to string and then to datetime object
    time_str = time_stamp.strftime("%Y-%m-%d %H:%M:%S")
    date_time = datetime.strptime(time_str, "%Y-%m-%d %H:%M:%S")

    # Adjust the time based on conditions
    if date_time.hour >= 23:
        date_time = date_time.replace(hour=23, minute=0, second=0)
    elif date_time.hour >= 11: 
        date_time = date_time.replace(hour=11, minute=0, second=0)
    else:
        date_time -= timedelta(days=1)  # Subtract a day
        date_time = date_time.replace(hour=23, minute=0, second=0)

    adjusted_times.append(date_time)  # Add adjusted time to list

data_output['Upravené časy intervalov'] = adjusted_times

closest_time_stamps = []
groups = data_output.groupby('Upravené časy intervalov')

# Loop through groups
for _, group in groups:

    # Get first time stamp in the group
    first_time_stamp = group.iloc[0]['Dátum a čas']
    closest_time_stamp = first_time_stamp

    # Iterate through rows in the group
    for _, row in group.iterrows():

        real_time_stamp = row['Dátum a čas']

        # Check if the absolute time difference is smaller than the current closest time stamp
        if abs((real_time_stamp - row['Upravené časy intervalov']).total_seconds()) < abs((closest_time_stamp - row['Upravené časy intervalov']).total_seconds()):
            closest_time_stamp = real_time_stamp
    
    closest_time_stamps.append(closest_time_stamp)

# Filter data_output to keep rows with closest_time_stamps
data_output = data_output[data_output['Dátum a čas'].isin(closest_time_stamps)]
data_output = data_output.reset_index(drop=True)

# Remove specified columns with NaN values
data_output = data_output.drop(['ISO Zhoda', 'ISO 0-5', 'ISO > 6,3'], axis=1)

In [13]:
path_to_15min_raw_data = 'data/PI_DataLink_Aglo_digital twins-5min-SP4-2018-15minute.xlsm'

data_15min = pd.read_excel(path_to_15min_raw_data)

data_15min

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,výstupné dáta,výstupné dáta.1,výstupné dáta.2,"výstupné dáta, látkové filtre - pred komínom","vstupné dáta, 3 stupen miesania, reguluje palič","vstupné dáta, reguluje palič",...,výstupné dáta.10,výstupné dáta.11,výstupné dáta.12,výstupné dáta.13,výstupné dáta.14,výstupné dáta.15,vstupné dáta.1,"vstupné dáta, reguluje palič.12",vstupné dáta.2,vstupné dáta.3
0,Datum od,2018-01-01 10:00:00,Datum do,2018-12-31 12:00:00,dopln do bunky h3 a dalsie doprava,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaT,NaN,NaT,dáta dostupné do r. 2019 kde boli uvedené do p...,NaN,NaN,NaN,"automatika miešacích bubnov (0=vypnutá, 1=zapn...",celkový prietok zmesi na SP4,...,NaN,NaN,NaN,NaN,NaN,"Teplota pod spekacím pásom, sacia komora 23. (...",Otáčky podávacieho bubna na začiatku SP,Rýchlosť spekaceho pásu,"Nefunkčné meranie od r 2019, nepoužívať","Nefunkčné meranie od r 2019, do roku 2019 sú data"
2,NaN,NaT,NaN,NaT,Sinter.EO4.O2.Comb.Gas.O2Content.%,Sinter.EO4.QSPAL.Comb.Gas.Volume.m3/hr,Sinter.EO4.TZL_P.Comb.Gas.TSPContent.mg/m3,Sinter.EO4.T_PRED.Comb.Gas.Input.Temp.C,Sinter.MB4.AUTO.MixingDrumNo4.AutomaticControl,Sinter.MB4.MS.Scales.Flow.Total.t/hr,...,Sinter.SP4.TS18.WindBoxNo18.Temperature.C,Sinter.SP4.TS19.WindBoxNo19.Temperature.C,Sinter.SP4.TS20.WindBoxNo20.Temperature.C,Sinter.SP4.TS21.WindBoxNo21.Temperature.C,Sinter.SP4.TS22.WindBoxNo22.Temperature.C,Sinter.SP4.TS23.WindBoxNo23.Temperature.C,Sinter.SP4.VOPB.FeedingDrum.Speed.ot/min,Sinter.SP4.VSP.SinterStrand.Speed.m/min,Sinter.TEX7.OTAC.Fan7.RPM,Sinter.TEX8.OTAC.Fan8.RPM
3,NaN,NaT,2018-01-01 10:00:00,2018-01-01 10:15:00,18.264017,700.266782,24.1226,0,1,228.719401,...,168.547434,176.211767,185.319134,202.393551,166.576,116.467117,0.7589,2.008533,1329.989014,1329.990013
4,NaN,NaT,2018-01-01 10:15:00,2018-01-01 10:30:00,18.2956,706.237531,24.534617,0,1,228.235471,...,165.095049,175.516066,176.825516,196.314184,165.274332,113.7811,0.751883,1.99735,1329.995504,1329.996503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34950,NaN,NaT,2018-12-31 10:45:00,2018-12-31 11:00:00,18.816999,708.57769,77.524151,0,1,227.62555,...,128.908483,128.110566,113.692766,164.451517,184.53605,154.147232,0.71165,2.0372,1360.043502,1360.002511
34951,NaN,NaT,2018-12-31 11:00:00,2018-12-31 11:15:00,18.816999,707.584498,80.058234,0,1,228.281668,...,132.293666,131.06595,120.070049,163.032982,176.629483,146.382766,0.711367,2.037433,1360.023505,1360.015007
34952,NaN,NaT,2018-12-31 11:15:00,2018-12-31 11:30:00,18.816999,708.969909,82.582333,0,1,228.045238,...,143.4212,146.387684,134.3365,184.606716,196.979168,158.829367,0.711333,2.0374,1360.039018,1360.010006
34953,NaN,NaT,2018-12-31 11:30:00,2018-12-31 11:45:00,18.816999,711.833297,76.172483,0,1,182.81985,...,149.56125,153.987266,144.052017,194.894467,204.016834,163.083466,0.6025,1.713833,1360.031012,1359.989502


In [14]:
path_to_30min_raw_data = 'data/PI_DataLink_Aglo_digital twins-5min-SP4-2018-30minute.xlsm'

data_30min = pd.read_excel(path_to_30min_raw_data)

data_30min

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,výstupné dáta,výstupné dáta.1,výstupné dáta.2,"výstupné dáta, látkové filtre - pred komínom","vstupné dáta, 3 stupen miesania, reguluje palič","vstupné dáta, reguluje palič",...,Unnamed: 195,Unnamed: 196,Unnamed: 197,Unnamed: 198,Unnamed: 199,Unnamed: 200,Unnamed: 201,Unnamed: 202,Unnamed: 203,Unnamed: 204
0,Datum od,2018-01-01 10:00:00,Datum do,2018-12-31 12:00:00,dopln do bunky h3 a dalsie doprava,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaT,NaN,NaT,dáta dostupné do r. 2019 kde boli uvedené do p...,NaN,NaN,NaN,"automatika miešacích bubnov (0=vypnutá, 1=zapn...",celkový prietok zmesi na SP4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaT,NaN,NaT,Sinter.EO4.O2.Comb.Gas.O2Content.%,Sinter.EO4.QSPAL.Comb.Gas.Volume.m3/hr,Sinter.EO4.TZL_P.Comb.Gas.TSPContent.mg/m3,Sinter.EO4.T_PRED.Comb.Gas.Input.Temp.C,Sinter.MB4.AUTO.MixingDrumNo4.AutomaticControl,Sinter.MB4.MS.Scales.Flow.Total.t/hr,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaT,2018-01-01 10:00:00,2018-01-01 10:30:00,18.279808,703.252156,24.328608,0,1,228.477436,...,Tag not found,Tag not found,Tag not found,Tag not found,Tag not found,Tag not found,Tag not found,Tag not found,Tag not found,Tag not found
4,NaN,NaT,2018-01-01 10:30:00,2018-01-01 11:00:00,18.11305,700.13979,26.645883,0,1,228.074851,...,Tag not found,Tag not found,Tag not found,Tag not found,Tag not found,Tag not found,Tag not found,Tag not found,Tag not found,Tag not found
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17474,NaN,NaT,2018-12-31 09:30:00,2018-12-31 10:00:00,18.816999,705.377481,75.309609,0,1,228.062101,...,Tag not found,Tag not found,Tag not found,Tag not found,Tag not found,Tag not found,Tag not found,Tag not found,Tag not found,Tag not found
17475,NaN,NaT,2018-12-31 10:00:00,2018-12-31 10:30:00,18.816999,705.049167,89.836509,0,1,227.973419,...,Tag not found,Tag not found,Tag not found,Tag not found,Tag not found,Tag not found,Tag not found,Tag not found,Tag not found,Tag not found
17476,NaN,NaT,2018-12-31 10:30:00,2018-12-31 11:00:00,18.816999,707.584468,77.117459,0,1,227.797426,...,Tag not found,Tag not found,Tag not found,Tag not found,Tag not found,Tag not found,Tag not found,Tag not found,Tag not found,Tag not found
17477,NaN,NaT,2018-12-31 11:00:00,2018-12-31 11:30:00,18.816999,708.277203,81.320284,0,1,228.163453,...,Tag not found,Tag not found,Tag not found,Tag not found,Tag not found,Tag not found,Tag not found,Tag not found,Tag not found,Tag not found


REMOVAL OF REDUNDANT DATA

In [15]:
data_15min = data_15min.iloc[3:, 2:]
data_15min = data_15min.reset_index(drop=True)

data_30min = data_30min.iloc[3:, 2:]
data_30min = data_30min.reset_index(drop=True)

# Drop rows with NaN values
data_15min.dropna(inplace=True)

data_30min.dropna(inplace=True)

MERGING BASED ON DATE AND TIME


- For 15 minutes data

In [53]:
# Convert date columns to datetime format
data_output['Date'] = pd.to_datetime(data_output['Upravené časy intervalov'])
data_15min['Date'] = pd.to_datetime(data_15min['Unnamed: 2'])

# Merge based on date intervals
result_data = pd.DataFrame(data_15min[data_15min['Unnamed: 2'].isin(data_output['Upravené časy intervalov'])])
result_data = result_data.sort_values(by='Unnamed: 2', ascending=False).reset_index(drop=True)

result_data['Zapis Merania'] = data_output['Dátum a čas']
result_data['Vzorka'] = data_output['Vzorka']
result_data['Rozsev 0-5'] = data_output['0-5']

# Rename columns
result_data = result_data.rename(columns={'Unnamed: 2': 'Interval od', 'Unnamed: 3': 'Interval do'})

# Drop unnecessary columns
result_data = result_data.drop('Date', axis=1)
result_data = result_data.drop('výstupné dáta, látkové filtre - pred komínom', axis=1)
result_data = result_data.drop('Unnamed: 12', axis=1)
result_data = result_data.drop('Unnamed: 13', axis=1)
result_data = result_data.drop('Unnamed: 14', axis=1)
result_data = result_data.drop('Unnamed: 16', axis=1)
result_data = result_data.drop('výstupné dáta.3', axis=1)
result_data = result_data.drop('výstupné dáta.4', axis=1)
result_data = result_data.drop('Unnamed: 27', axis=1)
result_data = result_data.drop('vstupné dáta, reguluje palič.10', axis=1)
result_data = result_data.drop('Unnamed: 30', axis=1)
result_data = result_data.drop('vstupné dáta.2', axis=1)
result_data = result_data.drop('vstupné dáta.3', axis=1)
result_data = result_data.drop('vstupné dáta, 3 stupeň', axis=1)

# Save to Excel
result_15min_data_path = 'data/parsed_15min_dataset.xlsx'
result_data.to_excel(result_15min_data_path, index=False)

In [54]:
result_data.count()
result_data

,Interval od,Interval do,výstupné dáta,výstupné dáta.1,výstupné dáta.2,"vstupné dáta, 3 stupen miesania, reguluje palič","vstupné dáta, reguluje palič","vstupné dáta, reguluje palič.1","vstupné dáta, reguluje palič.2",vstupné dáta,...,výstupné dáta.11,výstupné dáta.12,výstupné dáta.13,výstupné dáta.14,výstupné dáta.15,vstupné dáta.1,"vstupné dáta, reguluje palič.12",Zapis Merania,Vzorka,Rozsev 0-5
0,2018-12-31 11:00:00,2018-12-31 11:15:00,18.816999,707.584498,80.058234,1,228.281668,4.150883,4.769,6.86,...,131.06595,120.070049,163.032982,176.629483,146.382766,0.711367,2.037433,2018-12-31 12:08:18,41679,10.71
1,2018-12-30 23:00:00,2018-12-30 23:15:00,18.816999,707.283079,77.876267,1,226.86465,4.131533,4.769,6.86,...,142.0003,139.437549,204.874482,220.287483,200.544516,0.749917,2.152633,2018-12-30 23:56:11,41676,6.90
2,2018-12-30 11:00:00,2018-12-30 11:15:00,18.816999,695.341931,64.947766,1,227.116967,4.442967,4.769,6.86,...,132.520417,117.497401,166.51765,184.331782,161.278251,0.712333,2.08055,2018-12-30 12:43:11,41673,8.05
3,2018-12-29 23:00:00,2018-12-29 23:15:00,18.816999,706.556821,72.524867,1,228.063869,4.7744,4.769,6.86,...,152.835733,134.094401,177.335335,183.941018,147.457733,0.7289,2.113767,2018-12-30 00:15:03,41670,6.21
4,2018-12-29 11:00:00,2018-12-29 11:15:00,18.816999,710.190049,81.033282,1,227.547802,4.6434,4.769,6.86,...,151.898268,128.774884,170.230866,177.4666,147.93215,0.66065,1.824967,2018-12-29 12:23:16,41667,4.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,2018-01-03 11:00:00,2018-01-03 11:15:00,18.993284,715.636641,78.977666,0.863333,190.246086,2.574417,3.56085,7.290917,...,138.683849,168.911166,180.099066,156.663517,105.52515,0.702717,1.9422,2018-01-03 17:19:12,40007,7.31
497,2018-01-02 23:00:00,2018-01-02 23:15:00,18.356667,709.410084,61.53855,1,227.996302,3.473383,5.867217,8.040933,...,176.855198,204.440951,234.291532,208.389883,140.35035,0.748317,2.00315,2018-01-03 01:58:37,40006,7.04
498,2018-01-02 11:00:00,2018-01-02 11:15:00,19.169966,709.452829,161.9624,0,226.832703,3.276133,5.795817,8.048333,...,94.693117,112.700217,125.127517,105.031466,70.711683,0.765667,2.047617,2018-01-02 11:30:07,40004,36.17
499,2018-01-01 23:00:00,2018-01-01 23:15:00,17.924417,718.155392,31.431067,1,227.747202,3.567233,4.02235,8.63545,...,161.375434,186.621433,214.378767,186.251648,134.829633,0.761467,2.018433,2018-01-02 02:02:57,40003,11.62


- for 30 minutes data

In [35]:
# Convert date columns to datetime format

data_30min['Date'] = pd.to_datetime(data_30min['Unnamed: 2'])

# Merge based on date intervals
result_data = pd.DataFrame(data_15min[data_15min['Unnamed: 2'].isin(data_output['Upravené časy intervalov'])])
result_data.reset_index(drop=True)

result_data['Zapis Merania'] = data_output['Dátum a čas']
result_data['Vzorka'] = data_output['Vzorka']
result_data['Rozsev 0-5'] = data_output['0-5']

# Rename columns
result_data = result_data.rename(columns={'Unnamed: 2': 'Interval od', 'Unnamed: 3': 'Interval do'})

# Drop unnecessary columns
result_data = result_data.drop('Date', axis=1)
result_data = result_data.drop('výstupné dáta, látkové filtre - pred komínom', axis=1)
result_data = result_data.drop('Unnamed: 12', axis=1)
result_data = result_data.drop('Unnamed: 13', axis=1)
result_data = result_data.drop('Unnamed: 14', axis=1)
result_data = result_data.drop('Unnamed: 16', axis=1)
result_data = result_data.drop('výstupné dáta.3', axis=1)
result_data = result_data.drop('výstupné dáta.4', axis=1)
result_data = result_data.drop('Unnamed: 27', axis=1)
result_data = result_data.drop('vstupné dáta, reguluje palič.10', axis=1)
result_data = result_data.drop('Unnamed: 30', axis=1)
result_data = result_data.drop('vstupné dáta.2', axis=1)
result_data = result_data.drop('vstupné dáta.3', axis=1)
result_data = result_data.drop('vstupné dáta, 3 stupeň', axis=1)

# Save to Excel
result_30min_data_path = 'data/parsed_30min_dataset.xlsx'
result_data.to_excel(result_30min_data_path, index=False)

In [36]:
result_data.count()
result_data

,Interval od,Interval do,výstupné dáta,výstupné dáta.1,výstupné dáta.2,"vstupné dáta, 3 stupen miesania, reguluje palič","vstupné dáta, reguluje palič","vstupné dáta, reguluje palič.1","vstupné dáta, reguluje palič.2",vstupné dáta,...,výstupné dáta.11,výstupné dáta.12,výstupné dáta.13,výstupné dáta.14,výstupné dáta.15,vstupné dáta.1,"vstupné dáta, reguluje palič.12",Zapis Merania,Vzorka,Rozsev 0-5
4,2018-01-01 11:00:00,2018-01-01 11:15:00,18.245233,708.250231,26.974017,1,227.713533,3.7711,6.3795,5.953333,...,166.134733,196.306917,220.87265,192.920016,129.854734,0.740317,1.979583,2018-12-29 12:23:16,41667.0,4.21
52,2018-01-01 23:00:00,2018-01-01 23:15:00,17.924417,718.155392,31.431067,1,227.747202,3.567233,4.02235,8.63545,...,161.375434,186.621433,214.378767,186.251648,134.829633,0.761467,2.018433,2018-10-31 00:12:10,41309.0,7.47
100,2018-01-02 11:00:00,2018-01-02 11:15:00,19.169966,709.452829,161.9624,0,226.832703,3.276133,5.795817,8.048333,...,94.693117,112.700217,125.127517,105.031466,70.711683,0.765667,2.047617,2018-10-03 00:17:41,41145.0,8.17
148,2018-01-02 23:00:00,2018-01-02 23:15:00,18.356667,709.410084,61.53855,1,227.996302,3.473383,5.867217,8.040933,...,176.855198,204.440951,234.291532,208.389883,140.35035,0.748317,2.00315,2018-07-29 08:24:43,40759.0,9.76
196,2018-01-03 11:00:00,2018-01-03 11:15:00,18.993284,715.636641,78.977666,0.863333,190.246086,2.574417,3.56085,7.290917,...,138.683849,168.911166,180.099066,156.663517,105.52515,0.702717,1.9422,2018-07-12 08:40:09,40657.0,11.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34756,2018-12-29 11:00:00,2018-12-29 11:15:00,18.816999,710.190049,81.033282,1,227.547802,4.6434,4.769,6.86,...,151.898268,128.774884,170.230866,177.4666,147.93215,0.66065,1.824967,NaT,NaN,NaN
34804,2018-12-29 23:00:00,2018-12-29 23:15:00,18.816999,706.556821,72.524867,1,228.063869,4.7744,4.769,6.86,...,152.835733,134.094401,177.335335,183.941018,147.457733,0.7289,2.113767,NaT,NaN,NaN
34852,2018-12-30 11:00:00,2018-12-30 11:15:00,18.816999,695.341931,64.947766,1,227.116967,4.442967,4.769,6.86,...,132.520417,117.497401,166.51765,184.331782,161.278251,0.712333,2.08055,NaT,NaN,NaN
34900,2018-12-30 23:00:00,2018-12-30 23:15:00,18.816999,707.283079,77.876267,1,226.86465,4.131533,4.769,6.86,...,142.0003,139.437549,204.874482,220.287483,200.544516,0.749917,2.152633,NaT,NaN,NaN


ADJUSTING RESULT DATASET TO INFO DATASET

In [55]:
# Read the parsed_dataset excel file and select specific columns
info_dataset = pd.read_excel(result_15min_data_path)
info_dataset = info_dataset[['Interval od', 'Interval do', 'Zapis Merania', 'Vzorka', 'Rozsev 0-5', 'výstupné dáta.5']]

# Save the selected columns to a new excel file
info_dataset.to_excel('data/info_15min_dataset.xlsx', index=False)

In [56]:
info_dataset.count()
info_dataset

,Interval od,Interval do,Zapis Merania,Vzorka,Rozsev 0-5,výstupné dáta.5
0,2018-12-31 11:00:00,2018-12-31 11:15:00,2018-12-31 12:08:18,41679,10.71,10.393591
1,2018-12-30 23:00:00,2018-12-30 23:15:00,2018-12-30 23:56:11,41676,6.90,6.983189
2,2018-12-30 11:00:00,2018-12-30 11:15:00,2018-12-30 12:43:11,41673,8.05,7.814671
3,2018-12-29 23:00:00,2018-12-29 23:15:00,2018-12-30 00:15:03,41670,6.21,6.020195
4,2018-12-29 11:00:00,2018-12-29 11:15:00,2018-12-29 12:23:16,41667,4.21,5.159910
...,...,...,...,...,...,...
496,2018-01-03 11:00:00,2018-01-03 11:15:00,2018-01-03 17:19:12,40007,7.31,7.200983
497,2018-01-02 23:00:00,2018-01-02 23:15:00,2018-01-03 01:58:37,40006,7.04,8.146647
498,2018-01-02 11:00:00,2018-01-02 11:15:00,2018-01-02 11:30:07,40004,36.17,35.191028
499,2018-01-01 23:00:00,2018-01-01 23:15:00,2018-01-02 02:02:57,40003,11.62,10.935420


In [42]:
# Read the parsed_dataset excel file and select specific columns
info_dataset = pd.read_excel(result_30min_data_path)
info_dataset = info_dataset[['Interval od', 'Interval do', 'Zapis Merania', 'Vzorka', 'Rozsev 0-5', 'výstupné dáta.5']]

# Save the selected columns to a new excel file
info_dataset.to_excel('data/info_30min_dataset.xlsx', index=False)

In [ ]:
info_dataset.count()
info_dataset

ADJUSTING RESULT DATASET TO FINAL DATASET FOR NEURAL NETWORK

- 15 minutes dataset

In [58]:
final_dataset = pd.read_excel(result_15min_data_path)
final_dataset = final_dataset.drop(columns=['Interval od', 'Interval do', 'Zapis Merania', 'Vzorka', 'výstupné dáta.5'])

final_dataset.drop_duplicates(subset='Rozsev 0-5')
final_dataset.dropna(inplace=True)
final_dataset.reset_index(drop=True)

final_dataset_path = 'data/15min_dataset_pre_NN.xlsx'
final_dataset.to_excel(final_dataset_path, index=False)

In [59]:
final_dataset

,výstupné dáta,výstupné dáta.1,výstupné dáta.2,"vstupné dáta, 3 stupen miesania, reguluje palič","vstupné dáta, reguluje palič","vstupné dáta, reguluje palič.1","vstupné dáta, reguluje palič.2",vstupné dáta,"vstupné dáta, reguluje palič.3","vstupné dáta, reguluje palič.4",...,výstupné dáta.9,výstupné dáta.10,výstupné dáta.11,výstupné dáta.12,výstupné dáta.13,výstupné dáta.14,výstupné dáta.15,vstupné dáta.1,"vstupné dáta, reguluje palič.12",Rozsev 0-5
0,18.816999,707.584498,80.058234,1,228.281668,4.150883,4.769,6.86,3.53,-3.717500,...,136.321767,132.293666,131.065950,120.070049,163.032982,176.629483,146.382766,0.711367,2.037433,10.71
1,18.816999,707.283079,77.876267,1,226.864650,4.131533,4.769,6.86,3.53,-3.697167,...,137.372949,132.912383,142.000300,139.437549,204.874482,220.287483,200.544516,0.749917,2.152633,6.90
2,18.816999,695.341931,64.947766,1,227.116967,4.442967,4.769,6.86,3.67,-3.978633,...,141.632734,132.088500,132.520417,117.497401,166.517650,184.331782,161.278251,0.712333,2.080550,8.05
3,18.816999,706.556821,72.524867,1,228.063869,4.774400,4.769,6.86,3.81,-3.551733,...,141.585617,147.590018,152.835733,134.094401,177.335335,183.941018,147.457733,0.728900,2.113767,6.21
4,18.816999,710.190049,81.033282,1,227.547802,4.643400,4.769,6.86,3.90,-3.463567,...,171.417384,156.030383,151.898268,128.774884,170.230866,177.466600,147.932150,0.660650,1.824967,4.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,18.993284,715.636641,78.977666,0.863333,190.246086,2.574417,3.56085,7.290917,2.86,-4.387650,...,123.216516,156.741516,138.683849,168.911166,180.099066,156.663517,105.525150,0.702717,1.942200,7.31
497,18.356667,709.410084,61.538550,1,227.996302,3.473383,5.867217,8.040933,3.09,-4.315217,...,139.248766,177.915884,176.855198,204.440951,234.291532,208.389883,140.350350,0.748317,2.003150,7.04
498,19.169966,709.452829,161.962400,0,226.832703,3.276133,5.795817,8.048333,3.00,-4.903800,...,71.735000,101.088067,94.693117,112.700217,125.127517,105.031466,70.711683,0.765667,2.047617,36.17
499,17.924417,718.155392,31.431067,1,227.747202,3.567233,4.02235,8.63545,3.13,-4.577833,...,116.676800,161.837916,161.375434,186.621433,214.378767,186.251648,134.829633,0.761467,2.018433,11.62


- 30 minutes dataset

In [61]:
final_dataset = pd.read_excel(result_30min_data_path)
final_dataset = final_dataset.drop(columns=['Interval od', 'Interval do', 'Vzorka', 'výstupné dáta.5'])

final_dataset.drop_duplicates(subset='Rozsev 0-5')
final_dataset.dropna(inplace=True)
final_dataset.reset_index(drop=True)

final_dataset_path = 'data/30min_dataset_pre_NN.xlsx'
final_dataset.to_excel(final_dataset_path, index=False)

In [62]:
final_dataset

,výstupné dáta,výstupné dáta.1,výstupné dáta.2,"vstupné dáta, 3 stupen miesania, reguluje palič","vstupné dáta, reguluje palič","vstupné dáta, reguluje palič.1","vstupné dáta, reguluje palič.2",vstupné dáta,"vstupné dáta, reguluje palič.3","vstupné dáta, reguluje palič.4",...,výstupné dáta.10,výstupné dáta.11,výstupné dáta.12,výstupné dáta.13,výstupné dáta.14,výstupné dáta.15,vstupné dáta.1,"vstupné dáta, reguluje palič.12",Zapis Merania,Rozsev 0-5
0,18.245233,708.250231,26.974017,1,227.713533,3.771100,6.3795,5.953333,3.2200,-4.185000,...,165.478184,166.134733,196.306917,220.872650,192.920016,129.854734,0.740317,1.979583,2018-12-29 12:23:16,4.21
1,17.924417,718.155392,31.431067,1,227.747202,3.567233,4.02235,8.63545,3.1300,-4.577833,...,161.837916,161.375434,186.621433,214.378767,186.251648,134.829633,0.761467,2.018433,2018-10-31 00:12:10,7.47
2,19.169966,709.452829,161.962400,0,226.832703,3.276133,5.795817,8.048333,3.0000,-4.903800,...,101.088067,94.693117,112.700217,125.127517,105.031466,70.711683,0.765667,2.047617,2018-10-03 00:17:41,8.17
3,18.356667,709.410084,61.538550,1,227.996302,3.473383,5.867217,8.040933,3.0900,-4.315217,...,177.915884,176.855198,204.440951,234.291532,208.389883,140.350350,0.748317,2.003150,2018-07-29 08:24:43,9.76
4,18.993284,715.636641,78.977666,0.863333,190.246086,2.574417,3.56085,7.290917,2.8600,-4.387650,...,156.741516,138.683849,168.911166,180.099066,156.663517,105.525150,0.702717,1.942200,2018-07-12 08:40:09,11.97
5,18.448116,719.600217,59.185866,0,227.999701,2.560533,6.532083,8.2885,2.6800,-4.778600,...,171.102332,157.068233,195.104650,211.009634,175.816081,118.740151,0.724683,1.958350,2018-06-22 08:46:50,10.87
6,19.472666,698.403431,58.364583,1,167.874219,1.880533,3.186517,8.59555,2.6800,-3.682033,...,182.867133,125.495650,183.720382,167.336167,132.523000,94.436267,0.566567,1.453683,2018-05-30 01:37:22,9.27
7,18.586417,704.501627,42.122767,1,207.764134,1.962483,4.4024,8.108917,2.4900,-4.387100,...,167.749633,157.310401,191.877667,205.137767,171.594117,117.052717,0.654767,1.754467,2018-05-11 10:01:28,14.86
8,18.924416,434.164934,18.243000,0,0.000000,0.000000,2.276467,8.7166,2.5900,-1.886300,...,127.744118,129.563399,165.473234,170.306834,155.625250,97.064300,0.004317,0.000000,2018-04-21 01:26:41,8.33
9,18.610284,717.755596,91.464583,0,227.037050,2.450533,5.264983,7.526583,2.6000,-4.985967,...,119.661650,112.585899,144.681217,173.130801,157.136233,115.425400,0.737067,1.985933,2018-04-01 01:21:12,13.33
